
Content
This data set consists of three types of entities: 
    (a) the specification of an auto in terms of various characteristics,
    (b) its assigned insurance risk rating, 
    (c) its normalized losses in use as compared to other cars. The second rating corresponds to the degree to which
    the auto is more risky than its price indicates. Cars are initially assigned a risk factor symbol associated with its price.
    Then, if it is more risky (or less), this symbol is adjusted by moving it up (or down) the scale.
    Actuarians call this process "symboling". A value of +3 indicates that the auto is risky, -3 that it is probably pretty safe.

The third factor is the relative average loss payment per insured vehicle year.
This value is normalized for all autos within a particular size classification
(two-door small, station wagons, sports/speciality, etc...), and represents the average loss per car per year.

Note: Several of the attributes in the database could be used as a "class" attribute.

In [1]:
import pandas 
import numpy

import sklearn
from sklearn import model_selection # for splitting into train and test
from sklearn.preprocessing import StandardScaler,MinMaxScaler

from sklearn import linear_model # for logistic model



from scipy import stats
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
import matplotlib.pyplot as plt


In [2]:
pandas.options.display.max_columns = None     ## Columns
pandas.options.display.max_rows = None        ## Rows

In [3]:
df = pandas.read_csv('Automobile_data.csv')

In [4]:
df.shape

(205, 26)

In [5]:
# print(df.shape)



# missing_values = []
# for columns in df.columns:
#     if '?' in df[columns].unique():
#         missing_values.append(columns)
        

# missing_values

In [6]:
df = df.replace('?',numpy.NaN)    ## Introducting null value in place of '?'

In [7]:
df.isnull().sum()

symboling             0
normalized-losses    41
make                  0
fuel-type             0
aspiration            0
num-of-doors          2
body-style            0
drive-wheels          0
engine-location       0
wheel-base            0
length                0
width                 0
height                0
curb-weight           0
engine-type           0
num-of-cylinders      0
engine-size           0
fuel-system           0
bore                  4
stroke                4
compression-ratio     0
horsepower            2
peak-rpm              2
city-mpg              0
highway-mpg           0
price                 4
dtype: int64

## Imputation

In [8]:
# normalized-losses
df['normalized-losses'] = pandas.to_numeric(df['normalized-losses'])
df['normalized-losses'] = df['normalized-losses'].fillna(df['normalized-losses'].mean())

# num-of-doors
df['num-of-doors'] = df['num-of-doors'].replace('four',4)
df['num-of-doors'] = df['num-of-doors'].replace('two',2)
df['num-of-doors'] = df['num-of-doors'].fillna(df['num-of-doors'].mode()[0])

# bore
df['bore'] = pandas.to_numeric(df['bore'])
df['bore'] = df['bore'].fillna(df['bore'].mean())
#df.bore.value_counts()

# stroke
df['stroke'] = pandas.to_numeric(df['stroke'])
df['stroke'] = df['stroke'].fillna(df['stroke'].mean())


#horsepower
df['horsepower'] = pandas.to_numeric(df['horsepower'])
df['horsepower'] = df['horsepower'].fillna(df['horsepower'].mean())



#peak-rpm
df['peak-rpm'] = pandas.to_numeric(df['peak-rpm'])
df['peak-rpm'] = df['peak-rpm'].fillna(df['peak-rpm'].mean())


#num-of-cylinders
df['num-of-cylinders'] = df['num-of-cylinders'].replace('four',4)
df['num-of-cylinders'] = df['num-of-cylinders'].replace('six',6)
df['num-of-cylinders'] = df['num-of-cylinders'].replace('five',5)
df['num-of-cylinders'] = df['num-of-cylinders'].replace('two',2)
df['num-of-cylinders'] = df['num-of-cylinders'].replace('eight',8)
df['num-of-cylinders'] = df['num-of-cylinders'].replace('twelve',12)
df['num-of-cylinders'] = df['num-of-cylinders'].replace('three',3)


#price
df = df.dropna(axis=0, subset=['price'])    ## 4 rows deleted due to missing value in target

df['price'] = pandas.to_numeric(df['price'])

In [9]:
numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']

In [10]:
df.shape

(201, 26)

In [11]:
dfnumhot=pandas.get_dummies(df[['make','fuel-type','aspiration','body-style','drive-wheels','engine-location','engine-type','fuel-system']],
                                 prefix=None, prefix_sep='_', dummy_na=False, columns=None, sparse=False, drop_first=False, dtype=None)

dropped_column=['make','fuel-type','aspiration','body-style','drive-wheels','engine-location','engine-type','fuel-system']
df= df.drop(columns=dropped_column) 


## Concat
df = pandas.concat([df, dfnumhot], axis=1)
df.shape


(201, 68)

In [12]:
df.head(5)

,symboling,normalized-losses,num-of-doors,wheel-base,length,width,height,curb-weight,num-of-cylinders,engine-size,bore,stroke,compression-ratio,horsepower,peak-rpm,city-mpg,highway-mpg,price,make_alfa-romero,make_audi,make_bmw,make_chevrolet,make_dodge,make_honda,make_isuzu,make_jaguar,make_mazda,make_mercedes-benz,make_mercury,make_mitsubishi,make_nissan,make_peugot,make_plymouth,make_porsche,make_renault,make_saab,make_subaru,make_toyota,make_volkswagen,make_volvo,fuel-type_diesel,fuel-type_gas,aspiration_std,aspiration_turbo,body-style_convertible,body-style_hardtop,body-style_hatchback,body-style_sedan,body-style_wagon,drive-wheels_4wd,drive-wheels_fwd,drive-wheels_rwd,engine-location_front,engine-location_rear,engine-type_dohc,engine-type_l,engine-type_ohc,engine-type_ohcf,engine-type_ohcv,engine-type_rotor,fuel-system_1bbl,fuel-system_2bbl,fuel-system_4bbl,fuel-system_idi,fuel-system_mfi,fuel-system_mpfi,fuel-system_spdi,fuel-system_spfi
0,3,122.0,2.0,88.6,168.8,64.1,48.8,2548,4,130,3.47,2.68,9.0,111.0,5000.0,21,27,13495,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,1,0,0,0,0,0,0,1,1,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0
1,3,122.0,2.0,88.6,168.8,64.1,48.8,2548,4,130,3.47,2.68,9.0,111.0,5000.0,21,27,16500,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,1,0,0,0,0,0,0,1,1,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0
2,1,122.0,2.0,94.5,171.2,65.5,52.4,2823,6,152,2.68,3.47,9.0,154.0,5000.0,19,26,16500,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,1,0,0,0,0,1,1,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0
3,2,164.0,4.0,99.8,176.6,66.2,54.3,2337,4,109,3.19,3.40,10.0,102.0,5500.0,24,30,13950,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,1,0,0,1,0,1,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0
4,2,164.0,4.0,99.4,176.6,66.4,54.3,2824,5,136,3.19,3.40,8.0,115.0,5500.0,18,22,17450,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,1,0,1,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0


In [ ]:
###########################################################################################################################

In [37]:
# Split-out validation dataset
X = df.drop(columns=['price']).values
Y = df['price'].values
scaler = MinMaxScaler()
X=scaler.fit_transform(X)
X_Columns=df.select_dtypes(include=numerics).drop(columns=['price']).columns
validation_size = 0.30
seed = 100
X_train, X_test, Y_train, Y_test = sklearn.model_selection.train_test_split(X, Y, test_size=validation_size, random_state=seed)




In [61]:
from sklearn.linear_model import LinearRegression, Ridge, Lasso

# Create linear regression object
regr = linear_model.LinearRegression()

# Train the model using the training sets
regr.fit(X_train,Y_train)

# Make predictions using the testing set
y_pred_test = regr.predict(X_test)
y_pred_train = regr.predict(X_train)

print('____________________________________________________________________________\n\n')
print('TEST RESULTS\n')
# The coefficients
print('Coefficients: \n', regr.coef_)

print('\n')
# The mean squared error
print("Mean squared error: %.2f\n"
      % mean_squared_error(Y_test, y_pred_test))

# Explained variance score: 1 is perfect prediction
print('Variance score: %.2f\n' % r2_score(Y_test, y_pred_test))

print('r2:',r2_score(Y_test, y_pred_test) )
print('____________________________________________________________________________\n\n')

print('TRAIN RESULTS\n')

# The coefficients
print('Coefficients: \n', regr.coef_)
print('\n')
# The mean squared error
print("Mean squared error: %.2f\n"
      % mean_squared_error(Y_train, y_pred_train))

# Explained variance score: 1 is perfect prediction
print('Variance score: %.2f\n' % r2_score(Y_train, y_pred_train))
print('r2:',r2_score(Y_train, y_pred_train))

____________________________________________________________________________


TEST RESULTS

Coefficients: 
 [ 1.09396893e+03 -3.15219625e+02  6.98290682e+02  1.49033760e+04
 -1.05661440e+04  5.18099534e+03 -3.23224087e+03  1.01146175e+04
  7.63617875e+03  2.71262327e+04 -1.09660588e+03 -6.86043790e+03
 -7.89230028e+03  1.51323033e+03  5.94653144e+03 -1.22389184e+03
  8.60083615e+03 -5.75771331e+14 -5.75771331e+14 -5.75771331e+14
 -5.75771331e+14 -5.75771331e+14  1.20572635e+16 -5.75771331e+14
 -5.75771331e+14 -5.75771331e+14 -5.75771331e+14 -5.75771331e+14
 -5.75771331e+14 -5.75771331e+14 -5.75771331e+14 -5.75771331e+14
 -5.75771331e+14 -1.69710263e+15 -5.75771331e+14 -1.10950500e+16
 -5.75771331e+14 -5.75771331e+14 -5.75771331e+14 -9.88573079e+15
 -2.37693359e+16 -9.46852960e+15 -9.46852960e+15  2.47507793e+15
  2.47507793e+15  2.47507793e+15  2.47507793e+15  2.47507793e+15
 -3.09384509e+16 -3.09384509e+16 -3.09384509e+16  7.16734988e+15
 -3.35192882e+15 -6.52455180e+15 -6.52455180e+

In [62]:
regr = Ridge(alpha=0.05)

# Train the model using the training sets
regr.fit(X_train,Y_train)


# Make predictions using the testing set
y_pred_test = regr.predict(X_test)
y_pred_train = regr.predict(X_train)

print('____________________________________________________________________________\n\n')
print('TEST RESULTS\n')
# The coefficients
print('Coefficients: \n', regr.coef_)

print('\n')
# The mean squared error
print("Mean squared error for Ridge: %.2f\n"
      % mean_squared_error(Y_test, y_pred_test))

# Explained variance score: 1 is perfect prediction
print('Variance score for Ridge: %.2f\n' % r2_score(Y_test, y_pred_test))
print('r2 for Ridge:',r2_score(Y_test, y_pred_test))
print('____________________________________________________________________________\n\n')
print('TRAIN RESULTS\n')

# The coefficients
print('Coefficients: \n', regr.coef_)
print('\n')
# The mean squared error
print("Mean squared error for Ridge: %.2f\n"
      % mean_squared_error(Y_train, y_pred_train))

# Explained variance score: 1 is perfect prediction
print('Variance score for Ridge: %.2f\n' % r2_score(Y_train, y_pred_train))
print('r2 for Ridge:',r2_score(Y_train, y_pred_train))

____________________________________________________________________________


TEST RESULTS

Coefficients: 
 [ 1.66221136e+03 -2.40256301e+01  7.88759538e+02  1.17143312e+04
 -6.78695611e+03  5.81038598e+03 -2.03197222e+03  7.33797823e+03
  7.26655113e+03  1.78216278e+04 -8.79653900e+02 -5.23267799e+03
 -4.28237116e+03  6.67125269e+03  3.57236666e+03 -4.00190256e+02
  5.07316733e+03  2.03711692e+03  2.20603472e+03  5.86371038e+03
 -2.38315496e+03 -2.35601510e+03 -2.39535229e+02 -1.33825154e+03
  4.74436086e+03  7.69449819e+01  5.67137683e+03 -2.98873890e+03
 -2.61519977e+03 -1.25721996e+03 -7.55062365e+03 -2.99238455e+03
  4.75324422e+03  0.00000000e+00  2.27834672e+03 -3.03282416e+03
 -1.48791123e+03  6.17471647e+02 -6.74822973e+00  1.15861333e+03
 -1.15861333e+03 -1.13448060e+03  1.13448060e+03  1.00050886e+03
  1.04872297e+03 -9.54343209e+02 -6.03191963e+02 -4.91696666e+02
  4.65730727e+02 -7.87694590e+02  3.21963863e+02 -2.21268976e+03
  2.21268976e+03 -2.04729301e+03  2.44496366e+

In [63]:
regr = Lasso(alpha=0.05)

# Train the model using the training sets
regr.fit(X_train,Y_train)


# Make predictions using the testing set
y_pred_test = regr.predict(X_test)
y_pred_train = regr.predict(X_train)

print('____________________________________________________________________________\n\n')
print('TEST RESULTS\n')
# The coefficients
print('Coefficients: \n', regr.coef_)

print('\n')
# The mean squared error
print("Mean squared error for Lasso: %.2f\n"
      % mean_squared_error(Y_test, y_pred_test))

# Explained variance score: 1 is perfect prediction
print('Variance score for Lasso: %.2f\n' % r2_score(Y_test, y_pred_test))
print('r2 for Lasso:',r2_score(Y_test, y_pred_test))
print('____________________________________________________________________________\n\n')
print('TRAIN RESULTS\n')

# The coefficients
print('Coefficients: \n', regr.coef_)
print('\n')
# The mean squared error
print("Mean squared error for Lasso: %.2f\n"
      % mean_squared_error(Y_train, y_pred_train))

# Explained variance score: 1 is perfect prediction
print('Variance score for Lasso: %.2f\n' % r2_score(Y_train, y_pred_train))
print('r2 for Lasso:',r2_score(Y_train, y_pred_train))

____________________________________________________________________________


TEST RESULTS

Coefficients: 
 [ 1.09878570e+03 -1.20893384e+02  7.24945853e+02  1.49736063e+04
 -1.05367418e+04  5.19767457e+03 -3.00495063e+03  1.00195358e+04
  8.00901582e+03  2.60717287e+04 -7.33258158e+02 -6.85563365e+03
 -8.23531750e+03  2.20160253e+03  5.91813348e+03 -1.19224252e+03
  8.49793457e+03  4.63782908e+03  3.90747299e+03  6.62808942e+03
 -2.51448471e+03 -1.96921802e+03  0.00000000e+00 -2.61204952e+02
  3.76821471e+03  1.29238792e+03  4.98008693e+03 -1.75356948e+03
 -1.91191188e+03 -5.56257662e+00 -8.40785267e+03 -2.59999569e+03
  5.99576508e+03  0.00000000e+00  4.74056275e+03 -2.09678205e+03
 -3.76104614e+02  2.38940670e+03  4.57731086e+02  8.52849229e+03
 -0.00000000e+00 -3.13440234e+03  0.00000000e+00  7.94238856e+02
  1.36961109e+03 -7.36900194e+02 -2.75170201e+02  8.94868138e+01
  2.14392274e+01 -1.20874658e+03  0.00000000e+00 -4.10332513e+03
  8.20641077e-11 -1.29444901e+03  4.47362670e+

C:\Users\Jatin\Anaconda3\lib\site-packages\sklearn\linear_model\coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
